In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import subprocess

In [2]:
train_results_dir = os.getcwd() + "/../002-Training-models/train_results/"
!ls {train_results_dir}

model_yolov5_tf_no_imgsz_800_epochs_100_batch_16_dataset_v1_weapons
model_yolov5_tf_yolov5s_imgsz_640_epochs_100_batch_16_dataset_v1_weapons
model_yolov5_tf_yolov5s_imgsz_800_epochs_100_batch_16_dataset_v1_weapons
model_yolov7_tf_yolov7training_imgsz_640_epochs_100_batch_16_dataset_v1_weapons
model_yolov7_tf_yolov7training_imgsz_800_epochs_100_batch_8_dataset_v1_weapons
model_yolov8_tf_no_imgsz_800_epochs_100_batch_16_dataset_v1_weapons
model_yolov8_tf_yolov8l_imgsz_800_epochs_100_batch_16_dataset_v1_weapons
model_yolov8_tf_yolov8s_imgsz_640_epochs_100_batch_16_dataset_v1_weapons
model_yolov8_tf_yolov8s_imgsz_640_epochs_73_batch_16_dataset_v2
model_yolov8_tf_yolov8s_imgsz_800_epochs_100_batch_16_dataset_v1_weapons
model_yolov8_tf_yolov8s_imgsz_800_epochs_100_batch_16_dataset_v3_randomizedclips
model_yolov8_tf_yolov8s_imgsz_800_epochs_78_batch_16_dataset_v2


In [3]:
eval_results_dir = os.getcwd()
dataset_dir = os.getcwd()+"/../datasets/"
output_file = "output.txt"

In [4]:
# Download YOLOv8
!pip install ultralytics

from IPython import display
display.clear_output()

!mkdir yolov8

from ultralytics import YOLO
print('YOLOv8 setup complete.')

mkdir: yolov8: File exists
YOLOv8 setup complete.


In [5]:
# Download YOLOv7
# !git clone https://github.com/WongKinYiu/yolov7
%cd yolov7
!pip install -r requirements.txt

from IPython import display
display.clear_output()
print('YOLOv7 setup complete.')
%cd ..

YOLOv7 setup complete.
/Users/florenciavela/Desktop/flor/git/gun-detection/003-Evaluating-models


In [6]:
# Download YOLOv5
# !git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
# install dependencies as necessary
!pip install -qr requirements.txt  # install dependencies (ignore errors)
import torch

from IPython import display
display.clear_output()
print('YOLOv5 setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))
%cd ..

YOLOv5 setup complete. Using torch 2.0.0 CPU
/Users/florenciavela/Desktop/flor/git/gun-detection/003-Evaluating-models


In [17]:
%cd /Users/florenciavela/Desktop/flor/git/gun-detection/003-Evaluating-models

/Users/florenciavela/Desktop/flor/git/gun-detection/003-Evaluating-models


In [ ]:
for folder_name in sorted(os.listdir(train_results_dir)):
    if not folder_name.startswith("model"):
        continue

    folder_path = os.path.join(train_results_dir, folder_name, "weights")

    # Split the folder name into variables
    variables = folder_name.split('_')

    # Extract the required variables
    model = variables[1]
    tf = variables[3]
    imgsz = variables[5]
    epochs = variables[7]
    batch = variables[9]
    dataset = variables[11]
    
    print(folder_name)
    print("Model:", model)
    print("TF:", tf)
    print("Imgsz:", imgsz)
    print("Epochs:", epochs)
    print("Batch:", batch)
    print("Dataset:", dataset)
    print("Executing command...")
    print()
    
    if dataset == "v1":
        dataset_location = dataset_dir + model
    else:
        dataset_location = dataset_dir + model +"_guns_only"
        
    command = None
    
    if model == "yolov5":
        %cd yolov5
        best_pt_path = os.path.join(folder_path, "best.pt")
        
        command = f"python3 val.py --weights {best_pt_path} --data {dataset_location}/data.yaml --task val --name {folder_name}"
        print(command)

    elif model == "yolov7":
        %cd yolov7
        best_pt_path = os.path.join(folder_path, "best.pt")
        command = f"python3 test.py --weights {best_pt_path} --data {dataset_location}/data.yaml --task test --name {folder_name}"
        print(command)

    elif model == "yolov8":
        %cd yolov8
        best_pt_path = os.path.join(folder_path, "best.pt")
        command = f"yolo val model={best_pt_path} data={dataset_location}/data.yaml split=test name={folder_name}"
        print(command)

    if command:
        # Run the command using subprocess.Popen and capture the output
        with subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True) as process:
            # Open the output file for writing
            with open(f"{eval_results_dir}/{output_file}", "a") as file:
                !ls {eval_results_dir}/{output_file}
                file.write(folder_name)
                file.write(f"\nModel: {model}")
                file.write(f"\nTF: {tf}")
                file.write(f"\nImgsz: {imgsz}")
                file.write(f"\nEpochs: {epochs}")
                file.write(f"\nBatch: {batch}")
                file.write(f"\nDataset: {dataset}\n")
                # Read and display each line of the output as it becomes available
                for line in process.stdout:
                    print(line, end="")
                    file.write(line)
        %cd ..

    from IPython import display
    display.clear_output()
    

    print("Executing command...", "DONE!")
    print()

Executing command... DONE!

model_yolov8_tf_yolov8s_imgsz_800_epochs_78_batch_16_dataset_v2
Model: yolov8
TF: yolov8s
Imgsz: 800
Epochs: 78
Batch: 16
Dataset: v2
Executing command...

/Users/florenciavela/Desktop/flor/git/gun-detection/003-Evaluating-models/yolov8
yolo val model=/Users/florenciavela/Desktop/flor/git/gun-detection/003-Evaluating-models/../002-Training-models/train_results/model_yolov8_tf_yolov8s_imgsz_800_epochs_78_batch_16_dataset_v2/weights/best.pt data=/Users/florenciavela/Desktop/flor/git/gun-detection/003-Evaluating-models/../datasets/yolov8_guns_only/data.yaml split=test name=model_yolov8_tf_yolov8s_imgsz_800_epochs_78_batch_16_dataset_v2
/Users/florenciavela/Desktop/flor/git/gun-detection/003-Evaluating-models/output.txt
Ultralytics YOLOv8.0.149 🚀 Python-3.10.10 torch-2.0.0 CPU (Apple M2)
Model summary (fused): 168 layers, 11125971 parameters, 0 gradients

val: Scanning /Users/florenciavela/Desktop/flor/git/gun-detection/datasets/yolov8_guns_only/valid/labels.cac

In [ ]:
eval_results_dir = os.getcwd()
dataset_dir = os.getcwd()+"/../datasets/"
output_file = "output.txt"


def process_file(file_path):
    results = list()
    curr_res = dict()

    with open(file_path, 'r') as file:
        lines = file.readlines()

    for line in lines:
        if line.startswith("Model:"):
            curr_res["model"] = line.split()[1]
        elif line.startswith("TF:"):
            curr_res["tf"] = line.split()[1]
        elif line.startswith("Imgsz:"):
            curr_res["imgsz"] = line.split()[1]
        elif line.startswith("Epochs:"):
            curr_res["epochs"] = line.split()[1]
        elif line.startswith("Batch:"):
            curr_res["batch"] = line.split()[1]
        elif line.startswith("Dataset:"):
            curr_res["dataset"] = line.split()[1]
            print(line.split())
            
        if  "saved" in line or "all" in line or "knife" in line or "pistol" in line:
#             if line.strip(" ").startswith("all"):
#                 P, R, mAP5, mAP95 = line.split()[3:]
#                 curr_res["all_P"] = P
#                 curr_res["all_R"] = R
#                 curr_res["all_F1"] = (float(P) * float(R)) / (float(P) + float(R))
#                 curr_res["all_mAP@.5"] = mAP5
#                 curr_res["all_mAP@.5:.95"] = mAP95

#             elif line.strip(" ").startswith("knife"):
#                 P, R, mAP5, mAP95 = line.split()[3:]
#                 curr_res["knife_P"] = P
#                 curr_res["knife_R"] = R
#                 curr_res["knife_F1"] = (float(P) * float(R)) / (float(P) + float(R))
#                 curr_res["knife_mAP@.5"] = mAP5
#                 curr_res["knife_mAP@.5:.95"] = mAP95

            if line.strip(" ").startswith("pistol") or line.strip(" ").startswith("gun"):
                P, R, mAP5, mAP95 = line.split()[3:]
                curr_res["pistol_P"] = P
                curr_res["pistol_R"] = R
                curr_res["pistol_F1"] = (float(P) * float(R)) / (float(P) + float(R))
                curr_res["pistol_mAP@.5"] = mAP5
                curr_res["pistol_mAP@.5:.95"] = mAP95

            elif line.strip(" ").startswith("Results saved to"):
                model_key = line.split("/")[-1].replace('\x1b[0m', '').replace('\n', '')
                model_info = model_key.split("_")
                
#                 if not big_flag:
#                     curr_res["model"] = model_info[1]
#                     curr_res["tf"] = model_info[3]
#                     curr_res["imgsz"] = model_info[5]
#                     curr_res["epochs"] = model_info[7]
#                     curr_res["batch"] = model_info[9]
                    
                curr_res["model_key"] = model_key

                results.append(curr_res)
                curr_res = dict()

    df = pd.DataFrame(results)
    df.set_index('model_key', inplace=True)
    return df

def process_directory(directory_path):
    df_list = list()
    file_path = os.path.join(directory_path, output_file)
    df = process_file(file_path)

#     # Check for duplicate indexes
#     duplicate_indexes = temp.index.duplicated()

#     # Filter out rows with duplicate indexes
#     unique_df = temp[~duplicate_indexes]
#     return unique_df
    return df

df = process_directory(eval_results_dir)
df.to_csv(f'{results_dir}/output_file.csv')
df